In [79]:
import pandas as pd
import readFluvius
import datetime

In [80]:
def makeHourTable(din):
    df_out = pd.DataFrame(columns=["time", "afname", "injectie"])
    cnt = 0
    afname = 0
    injectie = 0
    for index, row in din.iterrows():
        van_datum = row["Van Datum"]
        van_uur = row['Van Tijdstip']
        register = row['Register']
        volume = row['Volume']
        [day, month, year] = [int(x) for x in van_datum.split('-')]
        [uur, min, sec] = [int(x) for x in van_uur.split(':')]
        afname = afname + volume if 'Afname' in register else afname
        injectie = injectie - volume if 'Injectie' in register else injectie
        cnt += 1
        #print(f'CNT {cnt}  {day}-{month}-{year} {uur}:{min}:{sec}  {register}  {volume}')
        if cnt == 8:
            van_tijd = datetime.datetime(year,month,day,uur)
            #tot_tijd = van_tijd + datetime.timedelta(hours=1)
            df_out.loc[len(df_out)] = [van_tijd, afname, injectie]
            afname = 0
            injectie = 0
            cnt = 0
    
    return df_out

In [81]:
reader = readFluvius.readFluvius('verbruik_full.csv')
fluvius = makeHourTable(reader.readData())

In [82]:
fluvius.to_excel("fluvius_full_uur.xlsx")

In [87]:
fluvius.columns

Index(['afname', 'injectie'], dtype='object')

In [88]:
fluvius

,afname,injectie
time,,
2023-06-23 00:00:00,0.179,0.000
2023-06-23 01:00:00,0.160,0.000
2023-06-23 02:00:00,0.167,0.000
2023-06-23 03:00:00,0.195,0.000
2023-06-23 04:00:00,0.209,0.000
...,...,...
2024-08-30 19:00:00,0.004,0.000
2024-08-30 20:00:00,0.581,-0.032
2024-08-30 21:00:00,1.402,0.000


In [89]:
#fluvius = pd.read_excel('fluvius_per_uur.xlsx')
belpex = pd.read_excel('belpex.xlsx')
#fluvius = fluvius.set_index(fluvius['time']).drop(columns=['time'])
belpex = belpex.set_index(belpex['time']).drop(columns=['time'])
m = pd.merge(fluvius, belpex, how='inner', left_index=True, right_index=True)
del belpex, fluvius

In [90]:
m

,afname,injectie,belpex
time,,,
2023-06-23 00:00:00,0.179,0.000,103.91
2023-06-23 01:00:00,0.160,0.000,95.20
2023-06-23 02:00:00,0.167,0.000,91.79
2023-06-23 03:00:00,0.195,0.000,91.28
2023-06-23 04:00:00,0.209,0.000,90.94
...,...,...,...
2024-08-30 19:00:00,0.004,0.000,133.46
2024-08-30 20:00:00,0.581,-0.032,119.11
2024-08-30 21:00:00,1.402,0.000,107.75


In [91]:
def calcAfname(belpex, volume):
    edag_excl = 0.1054 * belpex + 1.1200
    taks = 5.0329 + 0.2042
    distri = 4.59
    return (edag_excl + taks + distri) * 1.06 / 100 * volume

def calcInjectie(belpex, volume):
    return (0.0825 * belpex - 0.9550) * abs(volume) / 100

In [92]:
m['dyn_afname'] = m.apply(lambda row: calcAfname(row['belpex'], row['afname']), axis=1)
m['dyn_injectie'] = m.apply(lambda row: calcInjectie(row['belpex'], row['injectie']), axis=1)
m['var_afname'] = m['afname'].apply(lambda x: x * (0.0881 + 0.0459 + 0.050329 + 0.002042) * 1.06)
m['var_injectie'] = m['injectie'].apply(lambda x: abs(x) * 0.024)

In [93]:
m

,afname,injectie,belpex,dyn_afname,dyn_injectie,var_afname,var_injectie
time,,,,,,,
2023-06-23 00:00:00,0.179,0.000,103.91,0.041552,0.000000,0.035362,0.000000
2023-06-23 01:00:00,0.160,0.000,95.20,0.035584,0.000000,0.031609,0.000000
2023-06-23 02:00:00,0.167,0.000,91.79,0.036505,0.000000,0.032991,0.000000
2023-06-23 03:00:00,0.195,0.000,91.28,0.042514,0.000000,0.038523,0.000000
2023-06-23 04:00:00,0.209,0.000,90.94,0.045487,0.000000,0.041289,0.000000
...,...,...,...,...,...,...,...
2024-08-30 19:00:00,0.004,0.000,133.46,0.001061,0.000000,0.000790,0.000000
2024-08-30 20:00:00,0.581,-0.032,119.11,0.144735,0.002839,0.114778,0.000768
2024-08-30 21:00:00,1.402,0.000,107.75,0.331463,0.000000,0.276970,0.000000


In [94]:
m.to_excel("merged.xlsx")

In [95]:
calcAfname(100,1)

0.22776326000000005